In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("COVID").config("spark.some.config.option", "some-value").getOrCreate()
sc = SparkContext.getOrCreate()


pyspark.sql.SparkSession: Esta clase permite a los programadores programar en Spark con DataFrame y la funcionalidad SQL. SparkSession se usa para crear DataFrame, registrar DataFrame como tablas, tablas de caché, ejecuta SQL sobre tablas.

Como ya hemos dicho antes, a la hora de trabajar con un DataFrame, la base o de donde cuelga toda la funcionalidad es una SparkSession. Para crearla lo que haremos de la siguiente forma.

A través de SparkSession, puede crear un DataFrame, o puede registrar el DataFrame como una tabla y realizar una serie de operaciones SQL basadas en esto. DataFrame es similar a DataFrame en pandas.

In [ ]:
import re
from datetime import datetime
from array import array
from pyspark import SparkContext
from pyspark.sql import SQLContext, DataFrameWriter
import pyspark.sql.functions as Func
from pyspark.sql.types import IntegerType, StringType,DoubleType, FloatType
from pyspark.sql.functions import col,concat,udf,year,month,monotonically_increasing_id,dayofmonth,substring,regexp_replace
sqlContext = SQLContext(sc)

In [7]:
from array import array 

n = array('i',[1,2,3,4])

In [ ]:
# Catalogos Funciones
BNC_CAT = sqlContext.read.csv('/data/sandboxes/mcib/data/users/GTB/CIE/BNC_CATFunc.csv', header=True).withColumnRenamed('function_stm_id','function_stm_id1')
SIT_CAT = sqlContext.read.csv('/data/sandboxes/mcib/data/users/GTB/CIE/SIT_CATFunc.csv', header=True).withColumnRenamed('sender_application_id','sender_application_id1') 
H2H_CAT = sqlContext.read.csv('/data/sandboxes/mcib/data/users/GTB/CIE/H2H_CATFunc.csv', header=True).withColumnRenamed('function_channel_type','function_channel_type1') 

In [ ]:
data_path = '/data/master/mdco/data/engineering/t_mdco_tmidshst/'
h2h = sqlContext.read.parquet(data_path).filter(Func.col("load_date")>'2020-02-01')

In [ ]:
###cuando corras la consulta nueva usar la de h2h en lugar de unión y quitar contract channel id y quitar ,'GRUPO','AREA_GTB','EJECUTIVO_GTB'
func = ['I311','I312','I316','I313','I314','I315']
h2h_sel = h2h.select('commission_account_id','banking_name','transaction_date','source_branch_id',
                     'financial_cost_type','transaction_amount','option_catalog_name',
                     'function_channel_type','event_desc').where(
    (Func.ltrim(Func.rtrim(col('financial_cost_type'))) == "1")&((Func.ltrim(Func.rtrim(col('function_channel_type')))).isin(func)))
    #&((Func.ltrim(Func.rtrim(col('banking_name')))=='C&I MEXICO')|(Func.ltrim(Func.rtrim(col('banking_name'))).isNull())))

In [ ]:
h2h_sema = h2h_sel.withColumn('Semana_trans', Func.date_format(col('transaction_date'),'w')).withColumn(
            'Fecha_trans', Func.to_date(col('transaction_date'))).withColumn('Mes_trans',Func.month('transaction_date'))

In [ ]:
import pyspark
h2h.persist(pyspark.StorageLevel.MEMORY_ONLY)

In [ ]:
h2h.groupby('banking_name','Mes_trans').agg(Func.sum('monto').alias('sum'),Func.sum('trans').alias('count')
                                           ).sort('banking_name','Mes_trans').show(20,False)

In [ ]:
h2hf = h2h.where(col('banking_name')=='C&I MEXICO')

In [ ]:
h2hf = h2hf.join(H2H_CAT, Func.ltrim(Func.rtrim(col('function_channel_type1')))==Func.ltrim(Func.rtrim(col('function_channel_type'))),how='left').select('commission_account_id',
     'banking_name','source_branch_id','Fecha_trans','Semana_trans','Mes_trans','function_channel_type',col('H2H_DESC').alias('Desc'),'financial_cost_type','option_catalog_name','event_desc','monto','trans')

In [ ]:
h2hf.count()  #semana empieza en lunes y es la semana calendario #1785 antes   1830 el 27 de mayo 
#solo C&I MEXICO 10/06 : 996

In [ ]:
data_path = '/data/sandboxes/mcib/data/users/GTB'
h2hf.coalesce(1).write.mode('overwrite').format("com.databricks.spark.csv").option(
                    "header","true").save(data_path + '/h2h_inc.csv')

In [ ]:
#---------------------------------------------------------------------------------------------------------------------------

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import broadcast
from pyspark.sql.types import *

In [2]:
import re
from datetime import datetime
from array import array
from pyspark import SparkContext
from pyspark.sql import SQLContext, DataFrameWriter
import pyspark.sql.functions as Func
from pyspark.sql.types import IntegerType, StringType,DoubleType, FloatType
from pyspark.sql.functions import col,concat,udf,year,month,monotonically_increasing_id,dayofmonth,substring,regexp_replace

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
#una vez creada la session vamos a crear un dataframe a partir de un fichero de texto. 
df = spark.read.csv("C:/Users/URIEL/Documents/DATOS (CSV,XML,JSON,ETC)/planets.csv", header = True, inferSchema = True)

In [5]:
df.show()

+---------------+------+--------------+-----+--------+----+
|         method|number|orbital_period| mass|distance|year|
+---------------+------+--------------+-----+--------+----+
|Radial Velocity|     1|         269.3|  7.1|    77.4|2006|
|Radial Velocity|     1|       874.774| 2.21|   56.95|2008|
|Radial Velocity|     1|         763.0|  2.6|   19.84|2011|
|Radial Velocity|     1|        326.03| 19.4|  110.62|2007|
|Radial Velocity|     1|        516.22| 10.5|  119.47|2009|
|Radial Velocity|     1|        185.84|  4.8|   76.39|2008|
|Radial Velocity|     1|        1773.4| 4.64|   18.15|2002|
|Radial Velocity|     1|         798.5| null|   21.41|1996|
|Radial Velocity|     1|         993.3| 10.3|    73.1|2008|
|Radial Velocity|     2|         452.8| 1.99|   74.79|2010|
|Radial Velocity|     2|         883.0| 0.86|   74.79|2010|
|Radial Velocity|     1|         335.1| 9.88|   39.43|2009|
|Radial Velocity|     1|         479.1| 3.88|   97.28|2008|
|Radial Velocity|     3|        1078.0| 

In [6]:
df.filter(Func.col("method")=='Radial Velocity').show()

+---------------+------+--------------+-----+--------+----+
|         method|number|orbital_period| mass|distance|year|
+---------------+------+--------------+-----+--------+----+
|Radial Velocity|     1|         269.3|  7.1|    77.4|2006|
|Radial Velocity|     1|       874.774| 2.21|   56.95|2008|
|Radial Velocity|     1|         763.0|  2.6|   19.84|2011|
|Radial Velocity|     1|        326.03| 19.4|  110.62|2007|
|Radial Velocity|     1|        516.22| 10.5|  119.47|2009|
|Radial Velocity|     1|        185.84|  4.8|   76.39|2008|
|Radial Velocity|     1|        1773.4| 4.64|   18.15|2002|
|Radial Velocity|     1|         798.5| null|   21.41|1996|
|Radial Velocity|     1|         993.3| 10.3|    73.1|2008|
|Radial Velocity|     2|         452.8| 1.99|   74.79|2010|
|Radial Velocity|     2|         883.0| 0.86|   74.79|2010|
|Radial Velocity|     1|         335.1| 9.88|   39.43|2009|
|Radial Velocity|     1|         479.1| 3.88|   97.28|2008|
|Radial Velocity|     3|        1078.0| 

In [24]:
df.select('mass','distance','method').where(Func.ltrim(Func.rtrim('method')) == 'Radial Velocity').show()
#

+-----+--------+---------------+
| mass|distance|         method|
+-----+--------+---------------+
|  7.1|    77.4|Radial Velocity|
| 2.21|   56.95|Radial Velocity|
|  2.6|   19.84|Radial Velocity|
| 19.4|  110.62|Radial Velocity|
| 10.5|  119.47|Radial Velocity|
|  4.8|   76.39|Radial Velocity|
| 4.64|   18.15|Radial Velocity|
| null|   21.41|Radial Velocity|
| 10.3|    73.1|Radial Velocity|
| 1.99|   74.79|Radial Velocity|
| 0.86|   74.79|Radial Velocity|
| 9.88|   39.43|Radial Velocity|
| 3.88|   97.28|Radial Velocity|
| 2.53|   14.08|Radial Velocity|
| 0.54|   14.08|Radial Velocity|
| 1.64|   14.08|Radial Velocity|
|0.472|   15.36|Radial Velocity|
|  0.8|   12.53|Radial Velocity|
|0.165|   12.53|Radial Velocity|
| 3.53|   12.53|Radial Velocity|
+-----+--------+---------------+
only showing top 20 rows

